In [2]:
# Load the libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2, os
from glob import glob
from PIL import Image
from pathlib import Path

# Split the images and masks into smaller images

In [3]:
# Cut the plots into 500x500 image pieces

def cut_pieces(img_path, dims=500):
    img = np.load(img_path, allow_pickle=True)
    dirname = os.path.dirname(img_path)+"_PNG"
    basename = os.path.basename(img_path).replace("image.npy", "")
    if not os.path.exists(dirname):
        os.mkdir(dirname)
    shape = img.shape[:2]
    
    n_rows = int(shape[0]/dims)
    # check if image left has more than 250 pix
    rest_row = (shape[0]/dims) - n_rows
    n_cols = int(shape[1]/dims)
    rest_col = (shape[1]/dims) - n_cols
    #add the condition to add padding
    if ((n_rows == 0) or (rest_row > 0.5)):
        n_rows += 1
    if ((n_cols == 0) or (rest_col > 0.5)):
        n_cols +=1
    
    #cut and save the image
    count_im = 0
    for row in range(n_rows):        
        for column in range(n_cols):
            chunk = img[row*dims: (row+1)*dims,\
                        column*dims:(column+1)*dims,\
                        :]
            w, h , c = chunk.shape
            # pad the array
            chunk = np.pad(chunk,
                           ((0, 500-w), (0, 500-h), (0,0)),
                           mode='constant', constant_values=0)
            chunk1 = Image.fromarray(chunk)
            chunk1.save(f"{dirname}/{basename}image_{count_im}.png")
            count_im +=1
            
    # cut and save the respective mask
    count_m = 0
    mask = np.load(img_path.replace("image.npy", "weed_soil_crop_mask.npy"))
    for row in range(n_rows):        
        for column in range(n_cols):
            chunk = mask[row*dims: (row+1)*dims,\
                        column*dims:(column+1)*dims]
            w, h = chunk.shape
            chunk = np.pad(chunk,
                           ((0, 500-w), (0, 500-h)),
                           mode='constant', constant_values=0)
            chunk1 = Image.fromarray(chunk)
            chunk1.save(f"{dirname}/{basename}mask_{count_m}.png")
            count_m +=1

# Use this method as all my images are of a large resolution

In [3]:
img_path = Path("/group/pawsey0149/mmckay/singularity/raw_fullsize_images_masks/")
# Resize and cut
def resize_im(img_path,dims=224):
    img1 = np.load(img_path, allow_pickle=True)
    dirname = os.path.dirname(img_path)+"_PNG"
    basename = os.path.basename(img_path).replace("image.npy", "")
    if not os.path.exists(dirname):
        os.mkdir(dirname)
        
        
    # resize the image in 50%
    scale_percent = 50
    width = int(img1.shape[1] * scale_percent / 100)
    height = int(img1.shape[0] * scale_percent / 100)
    dim = (width, height)
    img = cv2.resize(img1, dim, interpolation = cv2.INTER_AREA)
    shape = img.shape[:2]
    
    n_rows = int(shape[0]/dims)
    # check if image left has more than 250 pix
    rest_row = (shape[0]/dims) - n_rows
    n_cols = int(shape[1]/dims)
    rest_col = (shape[1]/dims) - n_cols
    #add the condition to add padding
    if ((n_rows == 0) or (rest_row > 0.5)):
        n_rows += 1
    if ((n_cols == 0) or (rest_col > 0.5)):
        n_cols +=1
    
    #cut and save the image
    count_im = 0
    for row in range(n_rows):        
        for column in range(n_cols):
            chunk = img[row*dims: (row+1)*dims,\
                        column*dims:(column+1)*dims,\
                        :]
            w, h , c = chunk.shape
            # pad the array
            chunk = np.pad(chunk,
                           ((0, 224-w), (0, 224-h), (0,0)),
                           mode='constant', constant_values=0)
            chunk1 = Image.fromarray(chunk)
            chunk1.save(f"{dirname}/{basename}image_{count_im}.png")
            count_im +=1
            
    # MASK
    count_m = 0
    mask1 = np.load(img_path.replace("image.npy", "soil_crop_mask.npy"))
    # resize the image in 50%
    scale_percent = 50
    width = int(mask1.shape[1] * scale_percent / 100)
    height = int(mask1.shape[0] * scale_percent / 100)
    dim = (width, height)
    mask = cv2.resize(mask1, dim, interpolation = cv2.INTER_AREA)
    
    # cut and save the respective mask
    for row in range(n_rows):        
        for column in range(n_cols):
            chunk = mask[row*dims: (row+1)*dims,\
                        column*dims:(column+1)*dims]
            w, h = chunk.shape
            chunk = np.pad(chunk,
                           ((0, 224-w), (0, 224-h)),
                           mode='constant', constant_values=0)
            chunk1 = Image.fromarray(chunk)
            chunk1.save(f"{dirname}/{basename}mask_{count_m}.png")
            count_m +=1

In [4]:
#resize images
img_list = glob("/group/pawsey0149/mmckay/singularity/raw_fullsize_images_masks/*image.npy")
for m in img_list:
    resize_im(m)
    
#only ran this
#all masks appear as blank black images